# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch


In [45]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [46]:
# loading the dataset

train_df = pd.read_csv('/content/drive/MyDrive/q4/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/q4/test.csv')
labels_df = pd.read_csv('/content/drive/MyDrive/q4/test_labels.csv')
test_df = pd.merge(test_df, labels_df,on='id')
index_names = test_df[ (test_df['toxic'] == -1) & (test_df['severe_toxic'] == -1) & (test_df['severe_toxic'] == -1) & (test_df['obscene'] == -1) & (test_df['threat'] == -1)].index
test_df = test_df.drop(index_names,inplace=False)
#dataset understanding
train_df.sample(5)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
111598,5508f663f3fa9098,"""\n\n Your help desk request \n\nI've replied ...",0,0,0,0,0,0
8464,167f69436e467005,Furthermore Note 1 says it all (Housemates wer...,0,0,0,0,0,0
115034,671e0c56ec3e5dc6,I documented the change I made on Talk. The 'o...,0,0,0,0,0,0
103628,2a7038eabab79523,External links\nI've removed the list of exter...,0,0,0,0,0,0
145929,24566d3ba647e7a2,When are you going to learn that a definition ...,1,0,0,0,0,0


In [47]:
y_test_val = test_df[['toxic',	'severe_toxic',	'obscene',	'threat',	'insult',	'identity_hate']].values
y_train_val = train_df[['toxic',	'severe_toxic',	'obscene',	'threat',	'insult',	'identity_hate']].values
test_df = test_df[['id','comment_text']]

In [48]:
# let's see the total rows in train, test data and the numbers for the various categories
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
#print(train_df[cols_target].sum())


Total rows in test is 63978
Total rows in train is 159571


In [49]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [50]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [51]:
# train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, test_X.shape)


(159571,) (63978,)


Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [52]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [53]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [54]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]


In [55]:
#Obtain X_test and X_train 
bert_x_test = np.loadtxt('/content/drive/MyDrive/test_bert.txt')
bert_x_train = np.loadtxt('/content/drive/MyDrive/train_bert.txt')
use_x_train = np.loadtxt('/content/drive/MyDrive/train_use.txt')
use_x_test = np.loadtxt('/content/drive/MyDrive/test_use.txt')

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [56]:
!pip install scikit-multilearn

- Naive Bayes classifier on USE embedding

In [58]:

from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

classifier = BinaryRelevance(GaussianNB())
classifier.fit(use_x_train, y_train_val)
predictions = classifier.predict(use_x_test)
print('Accuracy:', accuracy_score(y_test_val, predictions))

Accuracy: 0.7983838194379318


- Naive Bayes classifier on BERT embedding

In [59]:

classifier = BinaryRelevance(GaussianNB())
classifier.fit(bert_x_train, y_train_val)
predictions = classifier.predict(bert_x_test)
print('Accuracy:', accuracy_score(y_test_val, predictions))

Accuracy: 0.7384413392103536


- Support Vector Machines on USE embedding

In [60]:

from sklearn.svm import LinearSVC

classifier = BinaryRelevance(LinearSVC(dual=False))
classifier.fit(use_x_train, y_train_val)
predictions = classifier.predict(use_x_test)
print('Accuracy:', accuracy_score(y_test_val, predictions))

Accuracy: 0.8930257275938603


- Support Vector Machines on BERT embedding

In [61]:

classifier = BinaryRelevance(LinearSVC(dual=False))
classifier.fit(bert_x_train, y_train_val)
predictions = classifier.predict(bert_x_test)
print('Accuracy:', accuracy_score(y_test_val, predictions))

Accuracy: 0.8893994810716184


Which classifier of the three performed better, and why do you think so ?

Ans:
In terms of accuracy, SVM outperformed the other two models. The essential premise of Naive Bayes is that all of the characteristics are independent of one another. SVM, on the other hand, seeks to identify some sort of relationship between features. 
 With strategies like One vs Rest and One vs One, the SVM may be employed and works quite well in Multilabel classification. The MLP, on the other hand, is ideal for singleclass categorization. For sklearn MLP,  performance will improve if different activations cannot be used on different layers. This allowed the SVM  to achieve slightly higher accuracy than the sklearn MLP.

#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [62]:
#MLP Model on USE Embedding

x_list = list()
for i in range(0, 6, 1):
  x_list.append(y_train_val.T[i].reshape(len(y_train_val), -1))


In [63]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed


def initialise_weights(input_shape):
  loc_val = 0.0
  scale_val = 0.01
  input_size = 100
  w1 = np.random.normal(loc=loc_val, scale = scale_val, size = (input_shape,input_size))
  w2 = np.random.normal(loc=loc_val, scale = scale_val, size = (input_size,1))
  return w1, w2
  

def loss_function(y_dash, y):
  # mse = np.mean((y_dash - y)**2,axis=0)
  # g_loss = (y_dash - y)
  return np.mean(np.square(y_dash - y),axis=0), (y_dash - y)


def sig_fn(x):
  """
    This is Sigmoid function
  """
  return (1 / (1 + np.exp((-1) * x)))


def diff_sig_fn(x):
  """
    This is derivative of sigmoid function
  """
  return np.multiply(sig_fn(x), 1 - sig_fn(x))


def forward_layer(w1, w2, X):
  """
    This Fn calculates output of the MLP using weights.
  """
  z1 = np.dot(X, w1)
  z2 = sig_fn(z1)
  y1 = np.dot(z2, w2)
  y2 = sig_fn(y1)
  return z2, y2


def backward_propogation(out1, dsig1, dsig2, g_loss, W2, X):
  """
    This Fn train the weights of MLP using its output value.
  """
  del_w2 = np.dot(out1.T , np.multiply(dsig2, g_loss))
  del_g = np.dot(np.multiply(dsig2, g_loss), W2.T)
  del_w1 = np.dot(X.T, np.multiply(dsig1, del_g))
  return del_w1, del_w2

In [69]:
def train(x_train, y_train):
  """
  This function trains the MLP and returns weights.

  @Return:
    Weights: tuple()
  """

    W1, W2 = initialise_weights(x_train.shape[1])

    lr = 0.07
    epochs = 25
    n = len(x_train)
    batch_size = 100
    
    for i in range(epochs):
        
        loss_list = []
        for j in range(0, n, batch_size):
            X = x_train[j:j+batch_size]
            Y = y_train[j:j+batch_size]
            out1, out2 = forward_layer(W1, W2, X)

            loss, g_loss = loss_function(out2, Y)
            loss_list.append(loss)

            dsig_1 = diff_sig_fn(out1)
            dsig_2 = diff_sig_fn(out2)

            del_w1, del_w2 = backward_propogation(out1, dsig_1, dsig_2, g_loss, W2, X)
            W1 = W1 - lr*del_w1
            W2 = W2 - lr*del_w2
        print(f"[ {i+1} / {epochs} ] loss: {np.mean(loss_list)}")
        # print("Loss: ", np.mean(loss_list))
    return W1, W2


def assign_class(y):
    for i in range(len(y)):
      if y[i] < 0.5:
        y[i] = 0
      else:
        y[i] = 1
    return y

def predict(w1, w2, x_test):
    z = sig_fn(np.dot(x_test, w1))
    y = sig_fn(np.dot(z, w2))
    return assign_class(y)
    # return y

#### MLP Model on USE Embedding

In [70]:

print("For Label 1:\n")
l1_w1, l1_w2 = train(use_x_train, x_list[0])
l1_y_pred = predict(l1_w1, l1_w2, use_x_test)

print("For Label 2:\n")
l2_w1, l2_w2 = train(use_x_train, x_list[1])
l2_y_pred = predict(l2_w1, l2_w2, use_x_test)

print("For Label 3:\n")
l3_w1, l3_w2 = train(use_x_train, x_list[2])
l3_y_pred = predict(l3_w1, l3_w2, use_x_test)

print("For Label 4:\n")
l4_w1, l4_w2 = train(use_x_train, x_list[3])
l4_y_pred = predict(l4_w1, l4_w2, use_x_test)

print("For Label 5:\n")
l5_w1, l5_w2 = train(use_x_train, x_list[4])
l5_y_pred = predict(l5_w1, l5_w2, use_x_test)

print("For Label 6:\n")
l6_w1, l6_w2 = train(use_x_train, x_list[5])
l6_y_pred = predict(l6_w1, l6_w2, use_x_test)


For Label 1:

[ 1 / 25 ] loss: 0.05888850998476255
[ 2 / 25 ] loss: 0.03818389502707432
[ 3 / 25 ] loss: 0.03761992230448517
[ 4 / 25 ] loss: 0.03731822130408709
[ 5 / 25 ] loss: 0.03710690231496893
[ 6 / 25 ] loss: 0.03694444430375018
[ 7 / 25 ] loss: 0.03681294980524407
[ 8 / 25 ] loss: 0.036702803461887284
[ 9 / 25 ] loss: 0.03660823106258714
[ 10 / 25 ] loss: 0.03652550967831609
[ 11 / 25 ] loss: 0.03645211181840317
[ 12 / 25 ] loss: 0.036386250352633355
[ 13 / 25 ] loss: 0.036326618173087036
[ 14 / 25 ] loss: 0.03627223115588284
[ 15 / 25 ] loss: 0.03622232950739792
[ 16 / 25 ] loss: 0.036176313689922815
[ 17 / 25 ] loss: 0.03613370158229061
[ 18 / 25 ] loss: 0.03609409907664409
[ 19 / 25 ] loss: 0.03605717940346743
[ 20 / 25 ] loss: 0.03602266826528095
[ 21 / 25 ] loss: 0.03599033292456513
[ 22 / 25 ] loss: 0.0359599740415934
[ 23 / 25 ] loss: 0.03593141946339167
[ 24 / 25 ] loss: 0.03590451942323612
[ 25 / 25 ] loss: 0.035879142777734296
For Label 2:

[ 1 / 25 ] loss: 0.01002975

In [71]:
#report accuracies on the test dataset 
result = np.concatenate((l1_y_pred, l2_y_pred, l3_y_pred, l4_y_pred, l5_y_pred, l6_y_pred), axis=1)
acc = accuracy_score(result, y_test_val)
print("Accuracy :", acc)

Accuracy : 0.899887461314827


#### MLP Model with BERT Embedding

In [74]:

x_list_bert = list()
for i in range(0, 6, 1):
  x_list_bert.append(y_train_val.T[i].reshape(len(y_train_val), -1))

In [75]:
# Using MLP on BERT embedding

print("For Label 1:\n")
l1_w1_bert, l1_w2_bert = train(bert_x_train, x_list_bert[0])
l1_y_pred_bert = predict(l1_w1_bert, l1_w2_bert, bert_x_test)


print("For Label 2:\n")
l2_w1_bert, l2_w2_bert = train(bert_x_train, x_list_bert[1])
l2_y_pred_bert = predict(l2_w1_bert, l2_w2_bert, bert_x_test)


print("For Label 3:\n")
l3_w1_bert, l3_w2_bert = train(bert_x_train, x_list_bert[2])
l3_y_pred_bert = predict(l3_w1_bert, l3_w2_bert, bert_x_test)


print("For Label 4:\n")
l4_w1_bert, l4_w2_bert = train(bert_x_train, x_list_bert[3])
l4_y_pred_bert = predict(l4_w1_bert, l4_w2_bert, bert_x_test)


print("For Label 5:\n")
l5_w1_bert, l5_w2_bert = train(bert_x_train, x_list_bert[4])
l5_y_pred_bert = predict(l5_w1_bert, l5_w2_bert, bert_x_test)


print("For Label 6:\n")
l6_w1_bert, l6_w2_bert = train(bert_x_train, x_list_bert[5])
l6_y_pred_bert = predict(l6_w1_bert, l6_w2_bert, bert_x_test)

For Label 1:

[ 1 / 25 ] loss: 0.055512617638638045


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp


[ 2 / 25 ] loss: 0.05582753834301355
[ 3 / 25 ] loss: 0.055860497766532353
[ 4 / 25 ] loss: 0.055869359291767444
[ 5 / 25 ] loss: 0.05586893894391055
[ 6 / 25 ] loss: 0.05587899355555013
[ 7 / 25 ] loss: 0.05589134821022003
[ 8 / 25 ] loss: 0.05591173395911775
[ 9 / 25 ] loss: 0.05589400752469836
[ 10 / 25 ] loss: 0.05590745431395836
[ 11 / 25 ] loss: 0.05590340467932812
[ 12 / 25 ] loss: 0.05591464794927425
[ 13 / 25 ] loss: 0.05591616388180174
[ 14 / 25 ] loss: 0.05592254799454017
[ 15 / 25 ] loss: 0.055917208665833086
[ 16 / 25 ] loss: 0.05590963326696509
[ 17 / 25 ] loss: 0.05591560858014584
[ 18 / 25 ] loss: 0.055913763318892584
[ 19 / 25 ] loss: 0.05590537767346804
[ 20 / 25 ] loss: 0.0559147808101102
[ 21 / 25 ] loss: 0.05591438121875329
[ 22 / 25 ] loss: 0.055908398115219665
[ 23 / 25 ] loss: 0.05591327180080136
[ 24 / 25 ] loss: 0.05591063655558854
[ 25 / 25 ] loss: 0.05590892673770055
For Label 2:

[ 1 / 25 ] loss: 0.00758343717622294
[ 2 / 25 ] loss: 0.0070886556858579365
[ 

In [76]:
#report accuracies on the test dataset 
result_bert = np.concatenate((l1_y_pred_bert, l2_y_pred_bert, l3_y_pred_bert, l4_y_pred_bert, l5_y_pred_bert, l6_y_pred_bert), axis=1)
acc_bert = accuracy_score(result_bert, y_test_val)
print("Accuracy :", acc_bert)

Accuracy : 0.8026040201319203


Now, compare your implemented MLP with the sklearn MLP layer.

In [78]:
#report accuracies both on train dataset and test dataset

from sklearn.neural_network import MLPClassifier

bert_mlp = MLPClassifier(hidden_layer_sizes=(32,16,6), activation='relu',solver='adam', max_iter=200)
bert_mlp.fit(bert_x_train, y_train_val)
predictions = bert_mlp.predict(bert_x_test)
print('Accuracy:', accuracy_score(y_test_val, predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy: 0.8789583919472319


Which MLP performed better? 
Understand and Analyse.

The MLP I created is more accurate. The reason may be that I  used different activation functions at different levels. In Sklearn MLP, the activation function remained the same at all layers 
 Since our MLP is just an MLP, sklearn's MLP performs better in certain environments. Sklearn MLP performs better with data that the  same activation was sufficient for all layers. 
 As a result, SkLearn's MLP performance improved, but predictive measurements were inadequate.